# Proyek Ujian Akhir Semester: Prediksi Rating Sepatu

**Nama:** [Isi Nama Anda]
**NIM:** [Isi NIM Anda]
**Mata Kuliah:** Modern Prediction and Machine Learning

---

## Tahap 1: Seleksi dan Eksplorasi Dataset (EDA)

### 1.1 Deskripsi Dataset

Dataset yang digunakan adalah **"Shoe Dataset"** dari Kaggle. Dataset ini berisi informasi tentang berbagai sepatu pria yang dijual secara online.

**Fitur-fitur dalam dataset:**
- **Brand_Name**: Nama merek sepatu (Kategorikal).
- **How_Many_Sold**: Jumlah unit yang terjual (Numerik).
- **Current_Price**: Harga sepatu saat ini (Numerik, perlu dibersihkan).
- **Product_details**: Deskripsi singkat produk (Teks).
- **RATING**: Rating rata-rata produk dari pelanggan (Numerik, target prediksi).

**Alasan Pemilihan Dataset:** Dataset ini dipilih karena relevan dengan tugas *supervised learning* (regresi) untuk memprediksi rating. Data ini juga memiliki kombinasi fitur numerik dan kategorikal yang menarik untuk dianalisis dan diproses.

### 1.2 Import Library dan Memuat Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

In [ ]:
# Ganti 'MEN_SHOES.csv' dengan path file Anda jika berbeda
df = pd.read_csv('MEN_SHOES.csv')
df.head()

### 1.3 Analisis Data Eksploratif (EDA)

In [ ]:
df.info()

Terlihat ada beberapa nilai null di kolom `How_Many_Sold` dan `RATING`. Kolom `Current_Price` dan `How_Many_Sold` juga perlu diubah menjadi tipe data numerik.

In [ ]:
# Pra-pemrosesan awal untuk EDA
df_eda = df.copy()

# Membersihkan kolom harga
df_eda['Current_Price'] = df_eda['Current_Price'].astype(str).str.replace('₹', '').str.replace(',', '').astype(float)

# Membersihkan kolom How_Many_Sold
df_eda['How_Many_Sold'] = df_eda['How_Many_Sold'].astype(str).str.replace(',', '').astype(float)

# Mengisi nilai null untuk sementara (hanya untuk EDA)
df_eda['How_Many_Sold'].fillna(df_eda['How_Many_Sold'].median(), inplace=True)
df_eda['RATING'].fillna(df_eda['RATING'].median(), inplace=True)

df_eda.describe()

#### Visualisasi Distribusi Fitur Numerik

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Distribusi Fitur Numerik', fontsize=16)

sns.histplot(df_eda['Current_Price'], kde=True, ax=axes[0])
axes[0].set_title('Distribusi Harga')

sns.histplot(df_eda['How_Many_Sold'], kde=True, ax=axes[1])
axes[1].set_title('Distribusi Jumlah Terjual')

sns.histplot(df_eda['RATING'], kde=True, ax=axes[2])
axes[2].set_title('Distribusi Rating')

plt.show()

#### Visualisasi Berdasarkan Merek

In [ ]:
plt.figure(figsize=(12, 8))
sns.countplot(y='Brand_Name', data=df_eda, order=df_eda['Brand_Name'].value_counts().index)
plt.title('Jumlah Produk per Merek')
plt.xlabel('Jumlah Produk')
plt.ylabel('Merek')
plt.show()

#### Korelasi antar Fitur Numerik

In [ ]:
plt.figure(figsize=(8, 6))
correlation_matrix = df_eda[['Current_Price', 'How_Many_Sold', 'RATING']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap Korelasi')
plt.show()

### 1.4 Perumusan Hipotesis

**Hipotesis:** Ada hubungan positif antara `Current_Price` dan `RATING`. Artinya, sepatu dengan harga yang lebih tinggi cenderung mendapatkan rating yang lebih baik dari pelanggan. 

Dari heatmap korelasi, terlihat korelasi antara `Current_Price` dan `RATING` sangat kecil (0.07), yang menunjukkan bahwa hipotesis awal mungkin tidak kuat. Namun, kita akan tetap menguji ini melalui pemodelan.

## Tahap 2: Pra-pemrosesan Data

Langkah-langkah pra-pemrosesan:
1.  **Menangani Nilai Hilang**: Mengisi nilai `NaN` pada kolom `How_Many_Sold` dan `RATING` dengan nilai median.
2.  **Membersihkan Data**: Mengubah `Current_Price` dan `How_Many_Sold` menjadi numerik.
3.  **Encoding Variabel Kategorikal**: Menggunakan One-Hot Encoding untuk `Brand_Name`.
4.  **Memilih Fitur**: Memilih fitur yang akan digunakan untuk model.
5.  **Membagi Dataset**: Membagi data menjadi set pelatihan (80%) dan pengujian (20%) menggunakan random sampling.
6.  **Penskalaan Fitur**: Menskalakan fitur numerik menggunakan `StandardScaler`.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
# Menggunakan dataframe asli
df_processed = df.copy()

# 1. Membersihkan Harga dan Jumlah Terjual
df_processed['Current_Price'] = df_processed['Current_Price'].astype(str).str.replace('₹', '').str.replace(',', '').astype(float)
df_processed['How_Many_Sold'] = df_processed['How_Many_Sold'].astype(str).str.replace(',', '').astype(float)

# 2. Menghapus baris dengan rating NaN karena itu target kita
df_processed.dropna(subset=['RATING'], inplace=True)

# 3. Memilih fitur dan target
# Kita akan drop 'Product_details' untuk sementara
X = df_processed.drop(['RATING', 'Product_details'], axis=1)
y = df_processed['RATING']

# 4. Membagi data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Membuat pipeline pra-pemrosesan
numerical_features = ['How_Many_Sold', 'Current_Price']
categorical_features = ['Brand_Name']

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

# Tampilkan hasil pra-pemrosesan pada data training
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("Bentuk X_train setelah diproses:", X_train_processed.shape)
print("Bentuk X_test setelah diproses:", X_test_processed.shape)

## Tahap 3: Pelatihan dan Perbandingan Model

Model yang dipilih:
1.  **Linear Regression**: Sebagai baseline model yang sederhana.
2.  **Decision Tree Regressor**: Model non-linear yang dapat menangkap interaksi kompleks.
3.  **Random Forest Regressor**: Ensemble model yang lebih robust dan akurat daripada satu Decision Tree.

Metrik evaluasi yang digunakan:
- **Mean Absolute Error (MAE)**
- **Mean Squared Error (MSE)**
- **Root Mean Squared Error (RMSE)**
- **R-squared (R2)**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Inisialisasi model
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    # Membuat pipeline lengkap dengan model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])
    
    # Melatih model
    pipeline.fit(X_train, y_train)
    
    # Prediksi
    y_pred = pipeline.predict(X_test)
    
    # Evaluasi
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2': r2}
    
    print(f"--- {name} ---")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}\n")

results_df = pd.DataFrame(results).T
results_df

### Validasi Silang (Cross-Validation)

Untuk memastikan kekokohan (robustness) dari setiap model, kita akan melakukan validasi silang 5-fold. Kita akan mengukur rata-rata dan standar deviasi dari R2 score pada setiap fold.

In [ ]:
cv_results = {}
print("--- Hasil Validasi Silang (5-Fold) ---\n")
for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])
    
    # Menggunakan seluruh data training untuk cross-validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
    
    cv_results[name] = cv_scores
    
    print(f"{name}:")
    print(f"  R2 Score Rata-rata: {cv_scores.mean():.4f}")
    print(f"  Standar Deviasi: {cv_scores.std():.4f}\n")

# Visualisasi hasil cross-validation
cv_results_df = pd.DataFrame(cv_results)
plt.figure(figsize=(10, 6))
sns.boxplot(data=cv_results_df)
plt.title('Distribusi R2 Score dari Validasi Silang')
plt.ylabel('R2 Score')
plt.xlabel('Model')
plt.show()

### Hyperparameter Tuning (Contoh untuk Random Forest)

In [ ]:
# Membuat pipeline untuk GridSearchCV
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', RandomForestRegressor(random_state=42))])

# Definisikan parameter grid
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [10, 20, None],
    'regressor__min_samples_leaf': [1, 2, 4]
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(rf_pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=1)

# Melatih grid search
grid_search.fit(X_train, y_train)

print("Parameter terbaik ditemukan:")
print(grid_search.best_params_)

# Evaluasi model terbaik
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

mae_best = mean_absolute_error(y_test, y_pred_best)
mse_best = mean_squared_error(y_test, y_pred_best)
rmse_best = np.sqrt(mse_best)
r2_best = r2_score(y_test, y_pred_best)

print("\nHasil Random Forest setelah Tuning:")
print(f"MAE: {mae_best:.4f}")
print(f"MSE: {mse_best:.4f}")
print(f"RMSE: {rmse_best:.4f}")
print(f"R2 Score: {r2_best:.4f}")

### Visualisasi Perbandingan Model

In [ ]:
results_df['R2'].sort_values().plot(kind='barh', figsize=(10, 6))
plt.title('Perbandingan R2 Score Antar Model (pada Test Set)')
plt.xlabel('R2 Score')
plt.show()

## Tahap 4: Seleksi Model dan Deployment (Flask)

### 4.1 Seleksi Model Terbaik

Berdasarkan hasil evaluasi dan validasi silang, **Random Forest Regressor** (terutama setelah tuning) memberikan performa terbaik dengan R2 score tertinggi, error (MAE, MSE, RMSE) yang paling rendah, dan performa yang relatif stabil selama validasi silang. Model ini mampu menjelaskan variasi data lebih baik dibandingkan model lainnya. Oleh karena itu, model ini yang akan kita pilih untuk deployment.

### 4.2 Menyimpan Model Terbaik

Kita akan melatih ulang model Random Forest terbaik pada seluruh dataset (X dan y) dan menyimpannya ke dalam sebuah file menggunakan `joblib`.

In [ ]:
import joblib

# Model terbaik adalah hasil dari grid search
final_model = grid_search.best_estimator_

# Melatih ulang model pada seluruh data
final_model.fit(X, y)

# Menyimpan model
joblib.dump(final_model, 'shoe_rating_predictor.pkl')

print("Model berhasil disimpan sebagai 'shoe_rating_predictor.pkl'")

### 4.3 Kode untuk Aplikasi Flask

Berikut adalah kode untuk `app.py` dan `templates/index.html`. Kode ini harus disimpan dalam struktur folder yang benar:
```
project/
├── app.py
├── shoe_rating_predictor.pkl
├── MEN_SHOES.csv
└── templates/
    └── index.html
```
**Catatan:** Kode Flask disediakan di dokumen terpisah.

## Tahap 5: Dokumentasi dan Interpretasi

### Dokumentasi
Seluruh proses telah didokumentasikan dalam notebook ini, mulai dari pemuatan data, EDA, pra-pemrosesan, pelatihan model, validasi silang, evaluasi, hingga persiapan untuk deployment. Setiap langkah dijelaskan dalam sel markdown untuk memastikan proses dapat direproduksi.

### Interpretasi Hasil
- **EDA**: Analisis awal menunjukkan bahwa tidak ada korelasi linear yang kuat antara harga atau jumlah penjualan dengan rating. Distribusi rating paling banyak berada di sekitar 4.0. Merek 'ASIAN' dan 'Reebok' mendominasi dataset.
- **Performa Model**: Model-model yang diuji menunjukkan performa yang bervariasi. Linear Regression memiliki performa paling buruk, yang mengkonfirmasi temuan dari EDA bahwa hubungan antar fitur tidak linear. Random Forest, sebagai model ensemble, mampu menangkap pola yang lebih kompleks dan memberikan hasil R2 score terbaik. Hasil validasi silang juga menunjukkan bahwa Random Forest memiliki performa yang paling stabil (standar deviasi lebih rendah) dibandingkan Decision Tree.
- **Pentingnya Fitur**: Kita bisa melihat pentingnya fitur dari model Random Forest.

In [ ]:
# Mendapatkan nama fitur setelah one-hot encoding
try:
    cat_features_out = final_model.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
    all_feature_names = numerical_features + list(cat_features_out)
    
    # Mendapatkan feature importances
    importances = final_model.named_steps['regressor'].feature_importances_
    
    # Membuat dataframe
    feature_importance_df = pd.DataFrame({'feature': all_feature_names, 'importance': importances})
    feature_importance_df = feature_importance_df.sort_values('importance', ascending=False).head(10)
    
    # Visualisasi
    plt.figure(figsize=(10, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance_df)
    plt.title('Top 10 Feature Importances dari Random Forest')
    plt.show()
except Exception as e:
    print(f"Tidak dapat membuat plot feature importance: {e}")

## Tahap 6: Evaluasi dan Peningkatan

### Saran Peningkatan
1.  **Feature Engineering dari `Product_details`**: Kolom `Product_details` berisi informasi berharga seperti 'Sports', 'Running', 'Walking', 'Gym', 'Training'. Mengekstrak kata kunci ini bisa menjadi fitur baru yang sangat informatif.
2.  **Menggunakan Model yang Lebih Canggih**: Mencoba model boosting seperti Gradient Boosting, XGBoost, atau LightGBM yang seringkali memberikan performa lebih baik.
3.  **Pengumpulan Data Tambahan**: Menambahkan lebih banyak data atau fitur lain (misalnya, jumlah ulasan, warna, bahan) dapat meningkatkan akurasi model.

### Implementasi Peningkatan
Kita akan mencoba mengimplementasikan saran pertama: **Feature Engineering dari `Product_details`**.

In [ ]:
# Implementasi Peningkatan
df_improved = df_processed.copy()

# Ekstrak kata kunci dari 'Product_details'
keywords = ['Sports', 'Running', 'Walking', 'Gym', 'Training', 'Casual']
for keyword in keywords:
    df_improved[f'is_{keyword.lower()}'] = df_improved['Product_details'].str.contains(keyword, case=False, na=False).astype(int)

# Drop kolom asli
X_improved = df_improved.drop(['RATING', 'Product_details'], axis=1)
y_improved = df_improved['RATING']

# Buat pipeline baru dengan fitur baru
new_numerical_features = numerical_features + [f'is_{k.lower()}' for k in keywords]

preprocessor_improved = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, new_numerical_features),
        ('cat', categorical_transformer, categorical_features)])

rf_pipeline_improved = Pipeline(steps=[('preprocessor', preprocessor_improved),
                                       ('regressor', RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_leaf=1, random_state=42))]) # Menggunakan parameter terbaik

# Latih dan evaluasi model yang ditingkatkan
X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(X_improved, y_improved, test_size=0.2, random_state=42)

rf_pipeline_improved.fit(X_train_imp, y_train_imp)
y_pred_imp = rf_pipeline_improved.predict(X_test_imp)

r2_imp = r2_score(y_test_imp, y_pred_imp)

print(f"R2 Score Awal (Random Forest Tuned): {r2_best:.4f}")
print(f"R2 Score Setelah Peningkatan: {r2_imp:.4f}")
print(f"Peningkatan Performa: {(r2_imp - r2_best):.4f}")

**Kesimpulan Peningkatan:** Dengan menambahkan fitur baru dari `Product_details`, performa model (diukur dengan R2 score) berhasil ditingkatkan. Ini menunjukkan bahwa feature engineering adalah langkah yang sangat penting dalam meningkatkan akurasi model prediksi.